# Introduction

We use 
* a single letter string to represent an **item**
* a `frozenset` to represent an **itemset**
* class `Transactions` to present the **transaction database** (a collection of **itemset**s)

## The `Transactions` class

In [1]:
# helper function
# we choose to use `frozenset` so that it is hashable (e.g., harder to remove duplicates); `set` is not
def itemset(*list_of_items): # `splat` allow us to send in multiple arguments
    return frozenset(sorted(list_of_items)) # there is no need to sort, but sorted set is better for debugging

# print(set) does not display set items in sorted order, hence we need to do it ourselves
# we overload the function depending on the type of the argument
def to_str(obj):
    if isinstance(obj, frozenset):
        return '{{{}}}'.format(', '.join(sorted(obj)))
    elif isinstance(obj, list):
        strs = [to_str(fs) for fs in obj]
        return '[ {} ]'.format(', '.join(strs))

In [2]:
t = itemset('B', 'A', 'D', 'X')
t

frozenset({'A', 'B', 'D', 'X'})

In [3]:
print(t)
print(to_str(t))

frozenset({'B', 'A', 'D', 'X'})
{A, B, D, X}


In [4]:
t1 = itemset('A', 'E')
list_of_itemsets = [t, t1]
to_str(list_of_itemsets)

'[ {A, B, D, X}, {A, E} ]'

In [5]:
itemset('A', 'B') == itemset('B', 'A')

True

In [6]:
import numpy as np

class transactions:
    def __init__(self, filename):
        trans_sets = []
        with open(filename, 'r') as f:
            for line in f:
                fields = line.split() # format: a transaction per line, consisting of items 
                trans_sets.append(itemset(*fields))
        self.trans_sets = trans_sets
    
    def get_tdb(self):
        return self.trans_sets

    def support(self, itemset):        
        # naive counting. `itemset` must be a set
        match_vector = [1 if itemset.issubset(trans) else 0 for trans in self.trans_sets]
        return np.sum(match_vector)
    
    def get_items(self):
        items = set()
        for itemset in self.trans_sets:
            items = items.union(itemset)
        return sorted(items)
    
    def get_frequent_items(self, minsup):
        return [itemset(item) for item in self.get_items() if self.support(itemset(item)) >= minsup]
    
    def len(self):
        return len(self.trans_sets)
    
    def dump(self):
        for trans in self.trans_sets:
            print('{}'.format(to_str(trans)))
    
        
db = transactions('asset/trans.txt')
db.dump()

my_itemset = itemset('B', 'A')
print('supp({}) = {}'.format(to_str(my_itemset), db.support(my_itemset)))
      
print('unique items are: {}'.format(db.get_items()))
for i in range(1, db.len()+1):
    print('minsup = {}, frequent items = {}'.format(i, to_str(db.get_frequent_items(i))))

{A, C, D}
{B, C}
{A, B, E}
{B, E}
{A, B, C, D, E}
{D, E}
supp({A, B}) = 2
unique items are: ['A', 'B', 'C', 'D', 'E']
minsup = 1, frequent items = [ {A}, {B}, {C}, {D}, {E} ]
minsup = 2, frequent items = [ {A}, {B}, {C}, {D}, {E} ]
minsup = 3, frequent items = [ {A}, {B}, {C}, {D}, {E} ]
minsup = 4, frequent items = [ {B}, {E} ]
minsup = 5, frequent items = [  ]
minsup = 6, frequent items = [  ]


## The Apriori Algorithm

In [7]:
def merge_itemsets(itemset1, itemset2):
    merged = set(itemset1)
    merged = merged.union(itemset2)
    return frozenset(merged)

In [8]:
itmset1 = itemset('A', 'B')
itmset2 = itemset('A', 'C')
itmset3 = itemset('A', 'D')
itmset4 = itemset('B', 'C')

print(to_str(merge_itemsets(itmset1, itmset2)))
print(to_str(merge_itemsets(itmset3, itmset4)))

{A, B, C}
{A, B, C, D}


In [9]:
# debug == 0: print nothing
# debug == 1: print input and output itemsets
# debug == 2: print all details
def gen_candidate_itemsets(freq_itemsets, debug = 1):
    if len(freq_itemsets) == 0: return []
    
    # freq_itemsets is a set of itemset (which is nothing but Set)  
    current_len = len(freq_itemsets[0])
    if debug >= 1: print('.. Input  (L_k)   :\t{}'.format(to_str(freq_itemsets)))
    
    # using set() to remove duplicates
    merged_itemsets = set([merge_itemsets(set1, set2) 
                           for set1 in freq_itemsets for set2 in freq_itemsets 
                               if str(set1) < str(set2)])
    if debug >= 2: print('..  .. Merged: {}'.format(to_str(merged_itemsets)))

    candidates = []
    for x in merged_itemsets:
        if debug >=2: print('.. .. Checking {}'.format(x))
        if len(x) == current_len + 1:
            pruned = False
            for elem in x:
                sub_itemset = set(x) # need to make a copy
                sub_itemset.remove(elem) # remove one item
                if sub_itemset not in freq_itemsets:
                    pruned = True
                    break
            if not pruned:
                candidates.append(x)
    if debug >= 1: print('.. Output (C_(k+1):\t{}'.format(to_str(candidates)))
    
    return candidates

In [10]:
my_itemsets = [itmset1, itmset2, itmset3, itmset4]
print(to_str(my_itemsets))
gen_candidate_itemsets(my_itemsets, debug = 1)

[ {A, B}, {A, C}, {A, D}, {B, C} ]
.. Input  (L_k)   :	[ {A, B}, {A, C}, {A, D}, {B, C} ]
.. Output (C_(k+1):	[ {A, B, C} ]


[frozenset({'A', 'B', 'C'})]

In [11]:
def output(freq_itemsets):
    for x in freq_itemsets:
        print('{} => {}'.format(to_str(x), db.support(x)))

def apriori(db, minsup):
    last_frequent_itemsets = db.get_frequent_items(minsup)
    output(last_frequent_itemsets)
    count = 1
    
    # generate candidate itemsets 
    candidate_itemsets = gen_candidate_itemsets(last_frequent_itemsets)
    while len(candidate_itemsets) > 0:
        count += 1
        print('\n** Iteration {}'.format(count))
        last_frequent_itemsets = [itemsets for itemsets in candidate_itemsets if db.support(itemsets) >= minsup]
        output(last_frequent_itemsets)
        candidate_itemsets = gen_candidate_itemsets(last_frequent_itemsets)


In [12]:
# change the default `debug` value to 1 (or 2) to see more info
apriori(db, 2)

{A} => 3
{B} => 4
{C} => 3
{D} => 3
{E} => 4
.. Input  (L_k)   :	[ {A}, {B}, {C}, {D}, {E} ]
.. Output (C_(k+1):	[ {B, C}, {B, D}, {A, C}, {A, D}, {C, D}, {C, E}, {B, E}, {A, E}, {D, E}, {A, B} ]

** Iteration 2
{B, C} => 2
{A, C} => 2
{A, D} => 2
{C, D} => 2
{B, E} => 3
{A, E} => 2
{D, E} => 2
{A, B} => 2
.. Input  (L_k)   :	[ {B, C}, {A, C}, {A, D}, {C, D}, {B, E}, {A, E}, {D, E}, {A, B} ]
.. Output (C_(k+1):	[ {A, D, E}, {A, B, C}, {A, C, D}, {A, B, E} ]

** Iteration 3
{A, C, D} => 2
{A, B, E} => 2
.. Input  (L_k)   :	[ {A, C, D}, {A, B, E} ]
.. Output (C_(k+1):	[  ]


If you turn on the debugging info, you can see that only `ABE` and `ACD` are identified as $C_3$. E.g., both `AB` and `AD` are in $L_2$, but since `BD` is not in $L_2$, `ABD` is not added to $C_3$. 

In [13]:
apriori(db, 3)

{A} => 3
{B} => 4
{C} => 3
{D} => 3
{E} => 4
.. Input  (L_k)   :	[ {A}, {B}, {C}, {D}, {E} ]
.. Output (C_(k+1):	[ {B, C}, {B, D}, {A, C}, {A, D}, {C, D}, {C, E}, {B, E}, {A, E}, {D, E}, {A, B} ]

** Iteration 2
{B, E} => 3
.. Input  (L_k)   :	[ {B, E} ]
.. Output (C_(k+1):	[  ]


In [14]:
apriori(db, 4)

{B} => 4
{E} => 4
.. Input  (L_k)   :	[ {B}, {E} ]
.. Output (C_(k+1):	[ {B, E} ]

** Iteration 2


In [15]:
apriori(db, 5)